In [ ]:
import numpy as np
import pandas as pd
import scipy
import os
import sys
import librosa
import copy
import pickle
from iteration_utilities import deepflatten
import matplotlib.pyplot as plt
import importlib
import glob
import warnings

ROOT = '' # set
sys.path.append(ROOT)

import utils

# change the width of the cells
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Spectrograms

## UMC

In [ ]:
%%time

show_bool = True # set

DATA = os.path.join(ROOT, 'data', 'UMC')
# pick the datasets
datasets = ['DKMP_OLD', 'DKMP_UMC', 'RKMP_OLD', 'RKMP_UMC']
# exclude noisy ids
exclude_noisy = ['ID_12', 'ID_14', 'ID_24', 'ID_004', 'ID_007', 'ID_013', 'ID_3']
# exclude bad
exclude_bad = ['ID_17', 'ID_18', 'ID_21'] # # ID_17 and ID_18 only have one class; dekomp and rekomp for ID_21 are the same recording!

# initialize the dictionaries
all_data = {'data': [], 
            'label': [],
            'frames':[],
            'wav': [],
            'id': [],
            'sig_qual': [],
            'excluded': []}

lens=[]
segs = []
specs_for_normalization = []
full_or_zeropad = ''
spec_len = 2 #seconds
fmin = 25
fmax = 1000
spec_frames = 128 # dimension
# spec_frames = 64 # dimension
# fmax = 400

if spec_frames == 128:
    # per channel means, used for normalization
    specs_mean = -71.84363555908203
    # per channel stds, used for normalization
    specs_std = 13.924535751342773
elif spec_frames == 64:
    # per channel means, used for normalization
    specs_mean = -58.466644287109375
    # per channel stds, used for normalization
    specs_std = 19.023942947387695
for dataset in datasets:
    print(f'Processing dataset {dataset}')
    # read the names of the recordings in the folder
    SEGMENTS = glob.glob(os.path.join(DATA, dataset, 'segments', '*.txt'))
    segments = [os.path.basename(SEG) for SEG in SEGMENTS]
    if dataset in ['DKMP_OLD', 'RKMP_OLD']:
        recs = [f"{rec.split('_')[0]}_{rec.split('_')[1]}" for rec in segments]
    elif dataset in ['DKMP_UMC', 'RKMP_UMC']:
        recs = [f"{rec.split('_')[0]}_{rec.split('_')[1]}_{rec.split('_')[2]}" for rec in segments]
    ids = [f"ID_{rec.split('_')[0]}" for rec in recs] 
    for rec, idx, SEG in zip(recs, ids, SEGMENTS):
        # detetmine the label
        if dataset in ['DKMP_OLD', 'DKMP_UMC']:
            label = 1
        elif dataset in ['RKMP_OLD', 'RKMP_UMC']:
            label = 0
        # determine the signal quality
        if idx in exclude_noisy:
            sig_qual = 0
        else:
            sig_qual = 1
        # determine if excluded
        if idx in exclude_bad:
            excluded = 0
        else:
            excluded = 1
        states = np.loadtxt(SEG)
        frames = np.where(states[:-1] != states[1:])[0]
        frames+=1
        states = [states[f] for f in frames]
        # find the start of each segment
        seg_starts = []
        for i, (frame, state) in enumerate(zip(frames, states)):
            # if cycle does not end abruptly
            if state == 1 and 1 in states[i+1:]:
                seg_states = states[i:i+4]
                if seg_states != [1, 2, 3, 4]:
                    raise Exception('Segment states are not correct!')
                seg_starts.append(i)
                seg_frames = frames[i:i+5] - frames[i] # include the start of the next segment, so we know the length of this segment
                all_data['label'].append(label)
                #all_data['frames'].append(seg_frames)
                all_data['wav'].append(rec)
                all_data['id'].append(idx)
                all_data['sig_qual'].append(sig_qual)
                all_data['excluded'].append(excluded)
        # save segments for each of the frequency bands         
        FREQ = os.path.join(DATA, dataset, f'raw')
        WAV = os.path.join(FREQ, f'{rec}.wav')
        y, sr = librosa.load(WAV, sr=None)
        # compute mel power spectrogram
        hop_length = int(sr*spec_len/spec_frames)
        mel_spectrogram = librosa.feature.melspectrogram(y=y, 
                                                     sr=sr, 
                                                     n_fft=hop_length*4, 
                                                     hop_length=hop_length, 
                                                     n_mels=spec_frames,
                                                     fmin=fmin, 
                                                     fmax=fmax
                                                        )
        # convert to dB scale
        mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
        # save to calculate normalization values
        specs_for_normalization.append(mel_spectrogram_db)
        #sigs_for_normalization.append(y)
        
        # normalize the spectrogram
        mel_spectrogram_db = (mel_spectrogram_db-specs_mean)/specs_std
        
        frames_spec = [round(f*mel_spectrogram_db.shape[1]/len(y)) for f in frames]
        
        for start in seg_starts:
            seg_states = states[start:start+4]
            seg_frames = frames[start:start+5] - frames[start]
            seg_frames_spec = np.array(frames_spec[start:start+5]) - frames_spec[start]
            # cut spec
            seg_y = y[frames[start]:frames[start+4]]
            # spectrogram segment
            spec = mel_spectrogram_db[:, frames_spec[start]:frames_spec[start+4]]
            # zero pad the timeseries
            if len(seg_y) > 8000:
                print(f'{rec} segment {start} is longer than 2000 samples')
            seg_y = copy.deepcopy(seg_y)
            seg_y.resize(8000)
            # zero pad spectrogram
            pad_size = ((0, 0), (0, max(0, spec_frames - spec.shape[1])))
            spec = np.pad(spec, pad_size, mode='constant')
            if show_bool:
                # Plot the spectrograms and timeseries with frames to see if everything is OK
                plt.figure(figsize=(4, 4))
                plt.imshow(spec)
                plt.axvline(x=seg_frames_spec[0], color = 'red')
                plt.axvline(x=seg_frames_spec[1], color = 'black')
                plt.axvline(x=seg_frames_spec[2], color = 'black')
                plt.axvline(x=seg_frames_spec[3], color = 'black')
                plt.yticks([])
                plt.xlim((0, len(spec[0])))
                plt.show()
                plt.close()
                plt.figure(figsize=(4, 4))
                plt.plot(seg_y)
                plt.axvline(x=seg_frames[0], color = 'red')
                plt.axvline(x=seg_frames[1], color = 'black')
                plt.axvline(x=seg_frames[2], color = 'black')
                plt.axvline(x=seg_frames[3], color = 'black')
                plt.yticks([])
                plt.xlim((0, len(seg_y)))
                plt.show()
                plt.close()
                print()
            all_data['data'].append(spec)
            all_data['frames'].append(seg_frames_spec)
            
# Sanity check for the normalization values
total_len = np.sum([len(x[0]) for x in specs_for_normalization])
specs_for_normalization_flattened = [arr.flatten() for arr in specs_for_normalization]
specs_for_normalization_flattened_combined = np.concatenate(specs_for_normalization_flattened)
print(f'Data spectrogram mean: {specs_mean}')
if np.mean(specs_for_normalization_flattened_combined) != specs_mean:
    print(f'WARNING: Data mean is not the same as defined: {np.mean(specs_for_normalization_flattened_combined)}')
    warnings.warn(f'WARNING: Data mean is not the same as defined: {np.mean(specs_for_normalization_flattened_combined)}')
print(f'Data spectrogram std: {specs_std}')
if np.std(specs_for_normalization_flattened_combined) != specs_std:
    print(f'WARNING: Data std is not the same as defined: {np.std(specs_for_normalization_flattened_combined)}')
    warnings.warn(f'WARNING: Data std is not the same as defined: {np.std(specs_for_normalization_flattened_combined)}')
    

DATASET = os.path.join(DATA, f'zbytes_UMC_dataset_spectrograms{spec_frames}.dat')
utils.dict2file(all_data, DATASET)
print(f'Dataset has been saved to {DATASET}')

## PhysioNet

In [ ]:
DATA = os.path.join(ROOT, 'data', 'physionet')

# list the test wavs
test_wavs = pd.read_csv(os.path.join(DATA, 'validation', 'REFERENCE.csv'), names=['rec', 'class'])['rec'].values
# train wavs used as 100% of the train data in our experiments
train_wavs = ['a0059', 'a0101', 'a0103', 'a0104', 'a0110', 'a0115', 'a0120', 'a0122', 'a0123', 'a0126', 'a0130', 'a0132', 'a0133', 'a0134', 'a0140', 'a0141', 'a0142', 'a0143', 'a0148', 'a0149', 'a0152', 'a0153', 'a0154', 'a0155', 'a0158', 'a0160', 'a0161', 'a0165', 'a0166', 'a0169', 'a0170', 'a0171', 'a0172', 'a0178', 'a0179', 'a0180', 'a0181', 'a0183', 'a0184', 'a0185', 'a0187', 'a0188', 'a0189', 'a0193', 'a0195', 'a0196', 'a0197', 'a0199', 'a0204', 'a0208', 'a0210', 'a0211', 'a0212', 'a0213', 'a0214', 'a0221', 'a0224', 'a0227', 'a0228', 'a0229', 'a0231', 'a0235', 'a0236', 'a0238', 'a0240', 'a0241', 'a0242', 'a0243', 'a0245', 'a0246', 'a0248', 'a0250', 'a0252', 'a0253', 'a0254', 'a0257', 'a0261', 'a0264', 'a0266', 'a0267', 'a0268', 'a0269', 'a0270', 'a0271', 'a0274', 'a0276', 'a0278', 'a0283', 'a0285', 'a0287', 'a0288', 'a0289', 'a0290', 'a0291', 'a0293', 'a0294', 'a0297', 'a0298', 'a0299', 'a0301', 'a0302', 'a0304', 'a0306', 'a0309', 'a0310', 'a0311', 'a0312', 'a0313', 'a0320', 'a0323', 'a0325', 'a0326', 'a0328', 'a0329', 'a0331', 'a0332', 'a0334', 'a0335', 'a0336', 'a0337', 'a0339', 'a0340', 'a0342', 'a0346', 'a0347', 'a0348', 'a0351', 'a0352', 'a0353', 'a0355', 'a0358', 'a0359', 'a0361', 'a0366', 'a0368', 'a0371', 'a0374', 'a0381', 'a0384', 'a0385', 'a0386', 'a0389', 'a0391', 'a0393', 'a0396', 'a0401', 'a0404', 'a0405', 'a0406', 'a0407', 'a0408', 'a0409', 'b0110', 'b0112', 'b0129', 'b0132', 'b0134', 'b0142', 'b0143', 'b0156', 'b0157', 'b0161', 'b0166', 'b0177', 'b0189', 'b0192', 'b0202', 'b0246', 'b0250', 'b0251', 'b0257', 'b0261', 'b0262', 'b0263', 'b0265', 'b0267', 'b0268', 'b0269', 'b0271', 'b0273', 'b0279', 'b0280', 'b0282', 'b0292', 'b0295', 'b0306', 'b0307', 'b0318', 'b0319', 'b0327', 'b0328', 'b0334', 'b0335', 'b0339', 'b0341', 'b0344', 'b0347', 'b0354', 'b0369', 'b0373', 'b0383', 'b0385', 'b0390', 'b0392', 'b0395', 'b0397', 'b0406', 'b0408', 'b0422', 'b0428', 'b0434', 'b0435', 'b0436', 'b0437', 'b0446', 'b0449', 'b0450', 'b0454', 'b0467', 'b0468', 'b0471', 'b0474', 'b0477', 'b0478', 'b0484', 'b0490', 'c0010', 'c0011', 'c0015', 'c0016', 'c0019', 'c0022', 'c0023', 'c0030', 'd0010', 'd0012', 'd0014', 'd0015', 'd0016', 'd0017', 'd0018', 'd0019', 'd0020', 'd0021', 'd0022', 'd0023', 'd0024', 'd0025', 'd0027', 'd0028', 'd0029', 'd0030', 'd0031', 'd0032', 'd0033', 'd0034', 'd0035', 'd0036', 'd0037', 'd0038', 'd0039', 'd0040', 'd0041', 'd0042', 'd0043', 'd0045', 'd0046', 'd0047', 'd0048', 'd0049', 'd0050', 'd0051', 'd0052', 'd0053', 'd0054', 'd0055', 'e00074', 'e00254', 'e00277', 'e00309', 'e00347', 'e00354', 'e00365', 'e00387', 'e00419', 'e00425', 'e00436', 'e00453', 'e00480', 'e00493', 'e00509', 'e00512', 'e00531', 'e00552', 'e00559', 'e00566', 'e00568', 'e00569', 'e00580', 'e00600', 'e00613', 'e00645', 'e00652', 'e00658', 'e00686', 'e00693', 'e00696', 'e00699', 'e00703', 'e00710', 'e00725', 'e00726', 'e00739', 'e00744', 'e00765', 'e00766', 'e00792', 'e00799', 'e00808', 'e00811', 'e00818', 'e00824', 'e00841', 'e00847', 'e00871', 'e00872', 'e00880', 'e00897', 'e00908', 'e00914', 'e00917', 'e00925', 'e00926', 'e00929', 'e00936', 'e00967', 'e00974', 'e00980', 'e00998', 'e01013', 'e01016', 'e01055', 'e01062', 'e01072', 'e01073', 'e01075', 'e01084', 'e01097', 'e01105', 'e01115', 'e01148', 'e01160', 'e01177', 'e01198', 'e01205', 'e01215', 'e01225', 'e01226', 'e01245', 'e01246', 'e01247', 'e01252', 'e01253', 'e01256', 'e01263', 'e01272', 'e01276', 'e01283', 'e01284', 'e01289', 'e01291', 'e01295', 'e01299', 'e01308', 'e01324', 'e01336', 'e01341', 'e01351', 'e01358', 'e01367', 'e01369', 'e01374', 'e01375', 'e01376', 'e01382', 'e01383', 'e01392', 'e01399', 'e01401', 'e01416', 'e01421', 'e01433', 'e01457', 'e01461', 'e01474', 'e01479', 'e01487', 'e01489', 'e01493', 'e01494', 'e01511', 'e01525', 'e01531', 'e01537', 'e01551', 'e01559', 'e01572', 'e01581', 'e01601', 'e01602', 'e01605', 'e01618', 'e01623', 'e01625', 'e01638', 'e01651', 'e01661', 'e01663', 'e01665', 'e01666', 'e01668', 'e01686', 'e01688', 'e01689', 'e01697', 'e01698', 'e01711', 'e01719', 'e01728', 'e01734', 'e01748', 'e01753', 'e01756', 'e01766', 'e01767', 'e01787', 'e01791', 'e01800', 'e01808', 'e01812', 'e01824', 'e01825', 'e01832', 'e01846', 'e01848', 'e01851', 'e01855', 'e01856', 'e01857', 'e01867', 'e01869', 'e01873', 'e01881', 'e01917', 'e01922', 'e01924', 'e01929', 'e01938', 'e01953', 'e01959', 'e01962', 'e01967', 'e01971', 'e01978', 'e02001', 'e02010', 'e02015', 'e02017', 'e02030', 'e02032', 'e02044', 'e02045', 'e02055', 'e02058', 'e02059', 'e02065', 'e02074', 'e02085', 'e02088', 'e02092', 'e02097', 'e02101', 'e02102', 'e02108', 'e02111', 'e02117', 'e02121', 'e02126', 'e02133', 'e02135', 'e02137', 'e02140', 'f0003', 'f0007', 'f0008', 'f0011', 'f0012', 'f0015', 'f0016', 'f0017', 'f0018', 'f0020', 'f0022', 'f0023', 'f0027', 'f0029', 'f0031', 'f0035', 'f0036', 'f0041', 'f0042', 'f0045', 'f0047', 'f0048', 'f0050', 'f0052', 'f0054', 'f0056', 'f0060', 'f0063', 'f0064', 'f0065', 'f0066', 'f0067', 'f0068', 'f0069', 'f0070', 'f0071', 'f0072', 'f0075', 'f0076', 'f0078', 'f0079', 'f0080', 'f0081', 'f0082', 'f0083', 'f0085', 'f0090', 'f0091', 'f0092', 'f0093', 'f0096', 'f0097', 'f0098', 'f0099', 'f0100', 'f0101', 'f0103', 'f0106', 'f0109', 'f0112', 'f0113', 'f0114']
# list the datasets
datasets = ['a', 'b', 'c', 'd', 'e', 'f']
# mean and std for normalization 
train_mean = -59.606563568115234
train_std = 15.96771240234375

In [ ]:
%%time

# initialize the dictionaries
train_data = {'data': [], 
              'label': [],
              'frames':[],
              'wav': [],
              'sig_qual': []}
test_data = {'data': [],
             'label': [],
             'frames':[],
             'wav': [],
             'sig_qual': []}

specs_for_normalization = []
sigs_for_normalization = []
lens=[]
segs = []
spec_len = 2.2 # seconds
fmin = 25
fmax = 1000
spec_frames = 128 # dimension
desired_frame_rate = spec_frames / spec_len  # 128 frames for every 2.2 seconds
for dataset in datasets:
    print(f'Processing dataset {dataset}')
    # read the reference file with the recording names, labels, and signal quality
    REF = os.path.join(DATA, 'annotations', 'updated', f'training-{dataset}', 'REFERENCE_withSQI.csv')
    ref_df = pd.read_csv(REF, names=['rec', 'class', 'sig_quality']) #class: 1 for abnormal and -1 for normal; sig_qual: 0 for bad quality 1 for good quality
    wavs = ref_df['rec']
    labels = ref_df['class']
    sig_quals = ref_df['sig_quality']
    for wav, label, sig_qual in zip(wavs, labels, sig_quals):
        if wav not in list(test_wavs) + train_wavs:
            continue
        # segmentations for 'e00001', 'e00032', 'e00039', 'e00044' are missing
        # change labels to 0 for normal and 1 for abnormal
        if label == -1:
            label = 0
        if sig_qual == 1:
            ANNOT = os.path.join(DATA, 'annotations', 'hand_corrected', f'training-{dataset}_StateAns') # hand corrected
            wav_annot = scipy.io.loadmat(os.path.join(ANNOT, f'{wav}_StateAns.mat'))
            wav_annot_states = wav_annot['state_ans']
        elif sig_qual == 0:
            ANNOT = os.path.join(DATA, 'annotations', 'springer_alg', f'training-{dataset}-Aut') # springer algorithm
            wav_annot = scipy.io.loadmat(os.path.join(ANNOT, f'{wav}_StateAns0.mat'))
            wav_annot_states = wav_annot['state_ans0']
        else:
            raise Exception('Signal quality has not been determined!') 
        # convert to pandas dataframe, then flatten
        df_wav_annot_states = pd.DataFrame(wav_annot_states, columns = ['frame', 'state'])
        frames = df_wav_annot_states['frame'].values
        frames = list(deepflatten(frames))
        fr = copy.deepcopy(frames)
        states = df_wav_annot_states['state']
        states = list(deepflatten(states, ignore=str))
        # find the start of each segment
        seg_starts = []
        for i, (frame, state) in enumerate(zip(frames, states)):  
            if state not in ['S1', 'systole', 'S2', 'diastole']:
                print(f'{wav}, {label=}, {sig_qual=}, {state=}, {fr[i]}')
            # if cycle does not end abruptly
            if state == 'S1' and 'S1' in states[i+1:]:
                seg_states = states[i:i+4]
                # skip if there is noise in this segment
                if 'N' in ''.join(seg_states):
                    print('Skip:')
                    print(f'\t{wav}, {label=}, {sig_qual=}, {state=}, {fr[i]}')
                    print(f'\t{seg_states}, frames:{fr[i:i+4]}')
                    continue
                if seg_states != ['S1', 'systole', 'S2', 'diastole']:
                    raise Exception('Segment states are not correct!')
                seg_starts.append(i)
                seg_frames = frames[i:i+5] - frames[i] # include the start of the next segment, so we know the length of this segment
        # to measure the lengths
        for start in seg_starts:
            seg_frames = frames[start:start+5] - frames[start]
            lens.append(seg_frames[-1]-seg_frames[0])
        # save segments
        FREQ = os.path.join(DATA, f'training-{dataset}', f'raw')
        WAV = os.path.join(FREQ, f'{wav}.wav')
        y, sr = librosa.load(WAV, sr=None)
        # compute mel power spectrogram
        hop_length = int(sr*spec_len/spec_frames)
        mel_spectrogram = librosa.feature.melspectrogram(y=y, 
                                                     sr=sr, 
                                                     n_fft=hop_length*4, 
                                                     hop_length=hop_length, 
                                                     n_mels=spec_frames,
                                                     fmin=25, 
                                                     fmax=1000
                                                        )
        # convert to dB scale
        mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
        # save to calculate normalization values
        if wav in train_wavs:
            specs_for_normalization.append(mel_spectrogram_db)
            sigs_for_normalization.append(y)
        # normalize the spectrogram
        mel_spectrogram_db = (mel_spectrogram_db-train_mean)/train_std
        #frames_spec = [frame/(sr*spec_len/spec_frames) for frame in frames] # CAREFULL, THIS PRODUCES WRONG RESULTS!!!
        frames_spec = [round(f*mel_spectrogram_db.shape[1]/len(y)) for f in frames]
        if wav in []:
            # Plot the spectrograms and timeseries with frames to see if everything is OK
            plt.figure(figsize=(20, 4))
            plt.imshow(mel_spectrogram_db)
            for start in seg_starts:
                plt.axvline(x=frames_spec[start], color = 'red')
                plt.axvline(x=frames_spec[start+1], color = 'black')
                plt.axvline(x=frames_spec[start+2], color = 'black')
                plt.axvline(x=frames_spec[start+3], color = 'black')
            plt.yticks([])
            plt.xlim((0, len(mel_spectrogram_db[0])))
            plt.show()
            plt.close()
            plt.figure(figsize=(20, 4))
            plt.plot(y)
            for start in seg_starts:
                plt.axvline(x=frames[start], color = 'red')
                plt.axvline(x=frames[start+1], color = 'black')
                plt.axvline(x=frames[start+2], color = 'black')
                plt.axvline(x=frames[start+3], color = 'black')
            plt.yticks([])
            plt.xlim((0, len(y)))
            plt.show()
            plt.close()
            print()
        for start in seg_starts:
            seg_states = states[start:start+4]
            seg_frames = frames[start:start+5] - frames[start]
            seg_frames_spec = np.array(frames_spec[start:start+5]) - frames_spec[start]
            # cut spec
            seg_y = y[frames[start]:frames[start+4]]
            # spectrogram segment
            spec = mel_spectrogram_db[:, frames_spec[start]:frames_spec[start+4]]
            # zero pad the timeseries
            if len(seg_y) > 4400:
                print(f'{wav} segment {start} is longer than 2500 samples')
            seg_y = copy.deepcopy(seg_y)
            seg_y.resize(4400)
            # zero pad spectrogram
            pad_size = ((0, 0), (0, max(0, spec_frames - spec.shape[1])))
            spec = np.pad(spec, pad_size, mode='constant')
            if wav in []:
                # Plot the spectrograms and timeseries with frames to see if everything is OK
                plt.figure(figsize=(4, 4))
                plt.imshow(spec)
                plt.axvline(x=seg_frames_spec[0], color = 'red')
                plt.axvline(x=seg_frames_spec[1], color = 'black')
                plt.axvline(x=seg_frames_spec[2], color = 'black')
                plt.axvline(x=seg_frames_spec[3], color = 'black')
                plt.yticks([])
                plt.xlim((0, len(spec[0])))
                plt.show()
                plt.close()
                plt.figure(figsize=(4, 4))
                plt.plot(seg_y)
                plt.axvline(x=seg_frames[0], color = 'red')
                plt.axvline(x=seg_frames[1], color = 'black')
                plt.axvline(x=seg_frames[2], color = 'black')
                plt.axvline(x=seg_frames[3], color = 'black')
                plt.yticks([])
                plt.xlim((0, len(seg_y)))
                plt.show()
                plt.close()
                print()
            if wav in test_wavs:
                test_data['data'].append(spec)
                test_data['frames'].append(seg_frames_spec)
                test_data['label'].append(label)
                test_data['wav'].append(wav)
                test_data['sig_qual'].append(sig_qual)
            elif wav in train_wavs:
                train_data['data'].append(spec)
                train_data['frames'].append(seg_frames_spec)
                train_data['label'].append(label)
                train_data['wav'].append(wav)
                train_data['sig_qual'].append(sig_qual)
                
# Sanity check for the normalization values
total_len = np.sum([len(x[0]) for x in specs_for_normalization])
specs_for_normalization_flattened = [arr.flatten() for arr in specs_for_normalization]
specs_for_normalization_flattened_combined = np.concatenate(specs_for_normalization_flattened)
print(f'Train data spectrogram mean: {train_mean}')
if np.mean(specs_for_normalization_flattened_combined) != train_mean:
    print(f'WARNING: Train mean is not the same as defined: {np.mean(specs_for_normalization_flattened_combined)}')
    warnings.warn(f'WARNING: Train mean is not the same as defined: {np.mean(specs_for_normalization_flattened_combined)}')
print(f'Train data spectrogram std: {train_std}')
if np.std(specs_for_normalization_flattened_combined) != train_std:
    print(f'WARNING: Train std is not the same as defined: {np.std(specs_for_normalization_flattened_combined)}')
    warnings.warn(f'WARNING: Train std is not the same as defined: {np.std(specs_for_normalization_flattened_combined)}')

In [ ]:
%%time

DATASET = os.path.join(DATA, f'zbytes_physionet_spectrograms{spec_frames}_dataset_selection.dat')
dataset = {'train':train_data, 'test':test_data}
utils.dict2file(dataset, DATASET)
print(f'Dataset has been saved to {DATASET}')

# Timeseries

## UMC

### UMC databuilder

In [ ]:
DATA = os.path.join(ROOT, 'data', 'UMC')

In [ ]:
# pick the datasets
datasets = ['DKMP_OLD', 'DKMP_UMC', 'RKMP_OLD', 'RKMP_UMC']
# select the frequency bands / channels
freq_bands = ['25-45', '45-80', '80-200', '200-400', '25-400']
# per channel means, used for normalization
pc_means = [-0.00070414954, -0.00070995715, -0.0015120364, -0.013083812, -0.00044722442]
# per channel stds, used for normalization
pc_stds = [0.10012293, 0.09927997, 0.097917296, 0.11611214, 0.09939657]
# exclude noisy ids
exclude_noisy = ['ID_12', 'ID_14', 'ID_24', 'ID_004', 'ID_007', 'ID_013', 'ID_3']
# exclude bad
exclude_bad = ['ID_17', 'ID_18', 'ID_21'] # # ID_17 and ID_18 only have one class; dekomp and rekomp for ID_21 are the same recording!

#### Zero-padding approach (one heart cycle per segment)

In [ ]:
%%time

# initialize the dictionaries
data_freq_bands = {}
for freq_band in freq_bands:
    data_freq_bands[freq_band] = []
all_data = {'data': copy.deepcopy(data_freq_bands), 
            'label': [],
            'frames':[],
            'wav': [],
            'id': [],
            'sig_qual': [],
            'excluded': []}

lens=[]
segs = []
full_or_zeropad = ''
for dataset in datasets:
    print(f'Processing dataset {dataset}')
    # read the names of the recordings in the folder
    SEGMENTS = glob.glob(os.path.join(DATA, dataset, 'segments', '*.txt'))
    segments = [os.path.basename(SEG) for SEG in SEGMENTS]
    if dataset in ['DKMP_OLD', 'RKMP_OLD']:
        recs = [f"{rec.split('_')[0]}_{rec.split('_')[1]}" for rec in segments]
    elif dataset in ['DKMP_UMC', 'RKMP_UMC']:
        recs = [f"{rec.split('_')[0]}_{rec.split('_')[1]}_{rec.split('_')[2]}" for rec in segments]
    ids = [f"ID_{rec.split('_')[0]}" for rec in recs]   
    for rec, idx, SEG in zip(recs, ids, SEGMENTS):
        # detetmine the label
        if dataset in ['DKMP_OLD', 'DKMP_UMC']:
            label = 0
        elif dataset in ['RKMP_OLD', 'RKMP_UMC']:
            label = 1
        # determine the signal quality
        if idx in exclude_noisy:
            sig_qual = 0
        else:
            sig_qual = 1
        # determine if excluded
        if idx in exclude_bad:
            excluded = 0
        else:
            excluded = 1
        states = np.loadtxt(SEG)
        frames = np.where(states[:-1] != states[1:])[0]
        frames+=1
        states = [states[f] for f in frames]
        # find the start of each segment
        frames = [f//4 for f in frames] # downsample from 4000 to 1000Hz
        seg_starts = []
        for i, (frame, state) in enumerate(zip(frames, states)):
            # if cycle does not end abruptly
            if state == 1 and 1 in states[i+1:]:
                seg_states = states[i:i+4]
                if seg_states != [1, 2, 3, 4]:
                    raise Exception('Segment states are not correct!')
                seg_starts.append(i)
                seg_frames = frames[i:i+5] - frames[i] # include the start of the next segment, so we know the length of this segment
                all_data['label'].append(label)
                all_data['frames'].append(seg_frames)
                all_data['wav'].append(rec)
                all_data['id'].append(idx)
                all_data['sig_qual'].append(sig_qual)
                all_data['excluded'].append(excluded)
        # save segments for each of the frequency bands         
        for freq_band, pc_mean, pc_std in zip(freq_bands, pc_means, pc_stds):
#             if freq_band != '25-400':
#                 continue
            FREQ = os.path.join(DATA, dataset, f'raw_filtBandIIR(ZP)4-{freq_band}_normRMS')
            WAV = os.path.join(FREQ, f'{rec}_filtBandIIR(ZP)4-{freq_band}_normRMS.wav')
            y, _ = librosa.load(WAV, sr = 4000)
            y_hat = librosa.resample(y=y, orig_sr=4000, target_sr=1000)
            # normalize the recording using mean and std of the train data that were calculated in advance and are now hardcoded
            y_hat = (y_hat-pc_mean) / pc_std
            c=0
            for start in seg_starts:
                #seg_states = states[start:start+4]
                seg_frames = frames[start:start+5] - frames[start]
                seg_y = y_hat[frames[start]:frames[start+4]]
                # zero pad the segments
                if len(seg_y) > 2000:
                    print(f'{rec} segment {start} is longer than 2000 samples')
                seg_y = copy.deepcopy(seg_y)
                seg_y.resize(2000)
                all_data['data'][freq_band].append(seg_y)

### Per channel normalization values

In [ ]:
means = []
stds = []
# Find mean and std of the train data of the NOT-ZERO-PADDED train data
for key in all_data['data']:
    freq_band = all_data['data'][key]
    freq_band = np.concatenate(freq_band).ravel()
    mean = np.mean(freq_band)
    means.append(mean)
    std = np.std(freq_band)
    stds.append(std)
print(f'Per channel means: {means}')
print(f'Per channel standard deviations: {stds}')

In [ ]:
%%time

DATASET = os.path.join(DATA, f'zbytes_UMC_dataset{full_or_zeropad}.dat')
dataset = all_data
utils.dict2file(dataset, DATASET)
print(f'Dataset has been saved to {DATASET}')

## PhysioNet

### PhysioNet databuilder

In [ ]:
DATA = os.path.join(ROOT, 'data', 'physionet')

In [ ]:
# list the test wavs
test_wavs = pd.read_csv(os.path.join(DATA, 'validation', 'REFERENCE.csv'), names=['rec', 'class'])['rec'].values
# pick the datasets
datasets = ['a', 'b', 'c', 'd', 'e', 'f']
# select the frequency bands / channels
freq_bands = ['25-45', '45-80', '80-200', '200-400', '400-600', '600-1000', '25-400', '25-1000']
# per channel means, used for normalization
pc_means = [-8.522174e-05, -9.561972e-05, -0.0001494191, -0.00080938824, -0.0025577587, -0.0001152527, -5.2299594e-05, -1.4092535e-05]
# per channel stds, used for normalization
pc_stds = [0.09962083, 0.09932303, 0.097970456, 0.095019236, 0.052084293, 0.004212678, 0.09908513, 0.06640719]

#### Multiple heart cycles per segment approach

In [ ]:
%%time

# initialize the dictionaries
data_freq_bands = {}
for freq_band in freq_bands:
    data_freq_bands[freq_band] = []
train_data = {'data': copy.deepcopy(data_freq_bands), 
              'label': [],
              'frames':[],
              'wav': [],
              'sig_qual': []}
test_data = {'data': copy.deepcopy(data_freq_bands),
             'label': [],
             'frames':[],
             'wav': [],
             'sig_qual': []}

segment_length = 2500
lens=[]
segs = []
full_or_zeropad = '_full'
for dataset in datasets:
    print(f'Processing dataset {dataset}')
    # read the reference file with the recording names, labels, and signal quality
    REF = os.path.join(DATA, 'annotations', 'updated', f'training-{dataset}', 'REFERENCE_withSQI.csv')
    ref_df = pd.read_csv(REF, names=['rec', 'class', 'sig_quality']) #class: 1 for abnormal and -1 for normal; sig_qual: 0 for bad quality 1 for good quality
    wavs = ref_df['rec']
    labels = ref_df['class']
    sig_quals = ref_df['sig_quality']
    for wav, label, sig_qual in zip(wavs, labels, sig_quals):
        ### segmentations for 'e00001', 'e00032', 'e00039', 'e00044' are missing
        # change labels to 0 for normal and 1 for abnormal
        if label == -1:
            label = 0
        if sig_qual == 1:
            ANNOT = os.path.join(DATA, 'annotations', 'hand_corrected', f'training-{dataset}_StateAns') # hand corrected
            wav_annot = scipy.io.loadmat(os.path.join(ANNOT, f'{wav}_StateAns.mat'))
            wav_annot_states = wav_annot['state_ans']
        elif sig_qual == 0:
            ANNOT = os.path.join(DATA, 'annotations', 'springer_alg', f'training-{dataset}-Aut') # springer algorithm
            wav_annot = scipy.io.loadmat(os.path.join(ANNOT, f'{wav}_StateAns0.mat'))
            wav_annot_states = wav_annot['state_ans0']
        else:
            raise Exception('Signal quality has not been determined!') 
        # convert to pandas dataframe, then flatten
        df_wav_annot_states = pd.DataFrame(wav_annot_states, columns = ['frame', 'state'])
        frames = df_wav_annot_states['frame'].values
        frames = list(deepflatten(frames))
        fr = copy.deepcopy(frames)
        frames = [x//2 for x in frames] # downsample from 2000 to 1000Hz
        states = df_wav_annot_states['state']
        states = list(deepflatten(states, ignore=str))
        # find the start of each segment
        FREQ = os.path.join(DATA, f'training-{dataset}', f'raw_filtBandIIR(ZP)4-25-400_normRMS')
        WAV = os.path.join(FREQ, f'{wav}_filtBandIIR(ZP)4-25-400_normRMS.wav')
        y, _ = librosa.load(WAV, sr = 2000)
        y = librosa.resample(y=y, orig_sr=2000, target_sr=1000)
        seg_starts = []
        seg_starts_frames = [] 
        for i, (frame, state) in enumerate(zip(frames, states)):  
            if i == 0 and state == 'S1':
                # skip as the first state is never full/is always clipped
                continue
            if state not in ['S1', 'systole', 'S2', 'diastole']:
                # print to see which states are "noise"
                print(f'{wav}, {label=}, {sig_qual=}, {state=}, {fr[i]}')
            if state == 'S1' and 'S1' in states[i+1:]:
                # the cycle starting here must not end abruptly (at least one whole cycle)
                if len(y[frame:])< segment_length:
                    # our segment is segment_length long
                    continue
                # find the index range of frames/segment that are included in that segment_length sample segment
                last_i = i
                for j in range(len(frames[i:])):
                    if frames[j+i]-frames[i] <= segment_length:
                        last_i = j+i
                    else:
                        break
                seg_states = states[i:last_i+1]
                # skip if there is noise in this segment
                if 'N' in ''.join(seg_states):
                    print('Skip:')
                    print(f'\t{wav}, {label=}, {sig_qual=}, {state=}, {fr[i]}')
                    print(f'\t{seg_states}, frames:{fr[i:last_i+1]}')
                    continue
                seg_starts.append(i)
                seg_frames = frames[i:last_i+1] - frames[i] # shift so they start with 0
                seg_frames = np.pad(seg_frames, (0, 28-len(seg_frames)), "constant", constant_values=-1) # 28 is max number of frames in our dataset
                seg_starts_frames.append(seg_frames)
                if wav in test_wavs:
                    test_data['frames'].append(seg_frames)
                    test_data['label'].append(label)
                    test_data['wav'].append(wav)
                    test_data['sig_qual'].append(sig_qual)
                else:
                    train_data['frames'].append(seg_frames)
                    train_data['label'].append(label)
                    train_data['wav'].append(wav)
                    train_data['sig_qual'].append(sig_qual)
        # save segments for each of the frequency bands         
        for freq_band, pc_mean, pc_std in zip(freq_bands, pc_means, pc_stds):
            FREQ = os.path.join(DATA, f'training-{dataset}', f'raw_filtBandIIR(ZP)4-{freq_band}_normRMS')
            WAV = os.path.join(FREQ, f'{wav}_filtBandIIR(ZP)4-{freq_band}_normRMS.wav')
            y, _ = librosa.load(WAV, sr = 2000)
            y = librosa.resample(y=y, orig_sr=2000, target_sr=1000)
            # normalize the recording using mean and std of the train data that were calculated in advance and are now hardcoded
            y = (y-pc_mean) / pc_std
            c=0
            for start, seg_frames in zip(seg_starts, seg_starts_frames):
                seg_y = y[frames[start]:frames[start]+segment_length]
                if wav in test_wavs:
                    test_data['data'][freq_band].append(seg_y)
                else:
                    train_data['data'][freq_band].append(seg_y)
                if freq_band == '25-400':
                    lens.append(len(seg_y))
                    segs.append([wav, start])

#### Zero-padding approach (one heart cycle per segment)

In [ ]:
%%time

# initialize the dictionaries
data_freq_bands = {}
for freq_band in freq_bands:
    data_freq_bands[freq_band] = []
train_data = {'data': copy.deepcopy(data_freq_bands), 
              'label': [],
              'frames':[],
              'wav': [],
              'sig_qual': []}
test_data = {'data': copy.deepcopy(data_freq_bands),
             'label': [],
             'frames':[],
             'wav': [],
             'sig_qual': []}

lens=[]
segs = []
full_or_zeropad = ''
for dataset in datasets:
    print(f'Processing dataset {dataset}')
#     if dataset != 'a':
#         continue
    # read the reference file with the recording names, labels, and signal quality
    REF = os.path.join(DATA, 'annotations', 'updated', f'training-{dataset}', 'REFERENCE_withSQI.csv')
    ref_df = pd.read_csv(REF, names=['rec', 'class', 'sig_quality']) #class: 1 for abnormal and -1 for normal; sig_qual: 0 for bad quality 1 for good quality
    wavs = ref_df['rec']
    labels = ref_df['class']
    sig_quals = ref_df['sig_quality']
    for wav, label, sig_qual in zip(wavs, labels, sig_quals):
        # segmentations for 'e00001', 'e00032', 'e00039', 'e00044' are missing
        # change labels to 0 for normal and 1 for abnormal
        if label == -1:
            label = 0
        if sig_qual == 1:
            ANNOT = os.path.join(DATA, 'annotations', 'hand_corrected', f'training-{dataset}_StateAns') # hand corrected
            wav_annot = scipy.io.loadmat(os.path.join(ANNOT, f'{wav}_StateAns.mat'))
            wav_annot_states = wav_annot['state_ans']
        elif sig_qual == 0:
            ANNOT = os.path.join(DATA, 'annotations', 'springer_alg', f'training-{dataset}-Aut') # springer algorithm
            wav_annot = scipy.io.loadmat(os.path.join(ANNOT, f'{wav}_StateAns0.mat'))
            wav_annot_states = wav_annot['state_ans0']
        else:
            raise Exception('Signal quality has not been determined!') 
        # convert to pandas dataframe, then flatten
        df_wav_annot_states = pd.DataFrame(wav_annot_states, columns = ['frame', 'state'])
        frames = df_wav_annot_states['frame'].values
        frames = list(deepflatten(frames))
        fr = copy.deepcopy(frames)
        frames = [x//2 for x in frames] # downsample from 2000 to 1000Hz
        states = df_wav_annot_states['state']
        states = list(deepflatten(states, ignore=str))
        # find the start of each segment
        seg_starts = []
        for i, (frame, state) in enumerate(zip(frames, states)):  
            if state not in ['S1', 'systole', 'S2', 'diastole']:
                print(f'{wav}, {label=}, {sig_qual=}, {state=}, {fr[i]}')
            # if cycle does not end abruptly
            if state == 'S1' and 'S1' in states[i+1:]:
                seg_states = states[i:i+4]
                # skip if there is noise in this segment
                if 'N' in ''.join(seg_states):
                    print('Skip:')
                    print(f'\t{wav}, {label=}, {sig_qual=}, {state=}, {fr[i]}')
                    print(f'\t{seg_states}, frames:{fr[i:i+4]}')
                    continue
                if seg_states != ['S1', 'systole', 'S2', 'diastole']:
                    raise Exception('Segment states are not correct!')
                seg_starts.append(i)
                seg_frames = frames[i:i+5] - frames[i] # include the start of the next segment, so we know the length of this segment
                if wav in test_wavs:
                    test_data['frames'].append(seg_frames)
                    test_data['label'].append(label)
                    test_data['wav'].append(wav)
                    test_data['sig_qual'].append(sig_qual)
                else:
                    train_data['frames'].append(seg_frames)
                    train_data['label'].append(label)
                    train_data['wav'].append(wav)
                    train_data['sig_qual'].append(sig_qual)
        # save segments for each of the frequency bands         
        for freq_band, pc_mean, pc_std in zip(freq_bands, pc_means, pc_stds):
#             if freq_band != '25-400':
#                 continue
            FREQ = os.path.join(DATA, f'training-{dataset}', f'raw_filtBandIIR(ZP)4-{freq_band}_normRMS')
            WAV = os.path.join(FREQ, f'{wav}_filtBandIIR(ZP)4-{freq_band}_normRMS.wav')
            y, _ = librosa.load(WAV, sr = 2000)
            y_hat = librosa.resample(y=y, orig_sr=2000, target_sr=1000)
            # normalize the recording using mean and std of the train data that were calculated in advance and are now hardcoded
            y_hat = (y_hat-pc_mean) / pc_std
            c=0
            for start in seg_starts:
                seg_states = states[start:start+4]
                seg_frames = frames[start:start+5] - frames[start]
                seg_y = y_hat[frames[start]:frames[start+4]]
                # zero pad the segments
                if len(seg_y) > 2500:
                    print(f'{wav} segment {start} is longer than 2500 samples')
                seg_y = copy.deepcopy(seg_y)
                seg_y.resize(2500)
                if wav in test_wavs:
                    test_data['data'][freq_band].append(seg_y)
                else:
                    train_data['data'][freq_band].append(seg_y)
                if freq_band == '25-400':
                    lens.append(len(seg_y))
                    segs.append([wav, start])

In [ ]:
%%time

DATASET = os.path.join(DATA, f'zbytes_physionet_dataset{full_or_zeropad}.dat')
dataset = {'train':train_data, 'test':test_data}
utils.dict2file(dataset, DATASET)
print(f'Dataset has been saved to {DATASET}')

### Per channel normalization values

In [ ]:
means = []
stds = []
# Find mean and std of the train data of the NOT-ZERO-PADDED train data
for key in dataset['train']['data']:
    freq_band = dataset['train']['data'][key]
    freq_band = np.concatenate(freq_band).ravel()
    mean = np.mean(freq_band)
    means.append(mean)
    std = np.std(freq_band)
    stds.append(std)
print(f'Per channel means: {means}')
print(f'Per channel standard deviations: {stds}')